In [1]:
from langchain.agents import tools
from langchain_experimental.utilities import PythonREPL

python_repl=PythonREPL()

In [2]:
python_repl.run("print(1+3)")

Python REPL can execute arbitrary code. Use with caution.


'4\n'

In [3]:
from langchain_openai import OpenAI,ChatOpenAI
import os

openai_api_key=os.getenv("OPEN_API_KEY")
model = ChatOpenAI(
    model='deepseek-chat', # 或者使用 'deepseek-reasoner' (对应 DeepSeek-R1)
    openai_api_key=openai_api_key,
    base_url='https://api.deepseek.com', # 必须修改此项，指向 DeepSeek 的服务器
    temperature=0.7
)

llm=model

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

promptFormat="""{query}
请根据上面的问题，生成Python代码计算出问题的答案，最后计算出来的结果用print()打印出来，请直接返回Python代码，不要返回其他任何内容的字符串。
"""

prompt=ChatPromptTemplate.from_template(promptFormat)

chain=prompt | llm | StrOutputParser()

In [5]:
result=chain.invoke({"query": "3箱苹果重45千克。一箱梨比一箱苹果多5千克，3箱梨重多少千克？"})
result


'```python\n# 已知3箱苹果重45千克\napple_weight_per_box = 45 / 3\n\n# 一箱梨比一箱苹果多5千克\npear_weight_per_box = apple_weight_per_box + 5\n\n# 计算3箱梨的重量\npear_total_weight = pear_weight_per_box * 3\n\nprint(int(pear_total_weight))\n```'

In [6]:
def parsePython(codeStr):
    codeStr=result.replace("```python","")
    codeStr=codeStr.replace("```","")
    return codeStr

In [7]:
chain=prompt | llm | StrOutputParser() | parsePython
chain

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}\n请根据上面的问题，生成Python代码计算出问题的答案，最后计算出来的结果用print()打印出来，请直接返回Python代码，不要返回其他任何内容的字符串。\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002537F0FD790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002537F57CED0>, root_client=<openai.OpenAI object at 0x000002537F04E190>, root_async_client=<openai.AsyncOpenAI object at 0x000002537F0FCF50>, model_name='deepseek-chat', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.deepseek.com')
| StrOutputParser()
| RunnableLambda(parsePython)

In [9]:
results=""
for chunk in chain.stream({"query": "3箱苹果重45千克。一箱梨比一箱苹果多5千克，3箱梨重多少千克？"}):
    print(chunk, end="", flush=True)
    results+=chunk


# 已知3箱苹果重45千克
apple_weight_per_box = 45 / 3

# 一箱梨比一箱苹果多5千克
pear_weight_per_box = apple_weight_per_box + 5

# 计算3箱梨的重量
pear_total_weight = pear_weight_per_box * 3

print(int(pear_total_weight))


In [10]:
python_repl.run(results)

'60\n'

In [11]:
chain=prompt | llm | StrOutputParser() | parsePython | python_repl.run

In [12]:
for chunk in chain.stream({"query": "3箱苹果重45千克。一箱梨比一箱苹果多5千克，3箱梨重多少千克？"}):
    print(chunk, end="", flush=True)

60
